# 동작구 Crawling 2: 관광지 목록을 불러와 네이버 지도에 주변 관광요소 검색하기 +Review수 검색

동작구 Crawling1에서 대한민국 구석구석을 크롤링해 관광지 목록을 txt파일로 저장하였다.

In [1]:
#패키지 import
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np

### Naver Map에서 관광지별 주변 환경을 크롤링하기

1. 네이버 지도에서 대중교통을 크롤링하는 crawlNaverMap()


2. 위의 정보 외에 추가적으로 파악하고자 하는 주변 음식점, 카페, 문화시설, 숙박시설, 공원, 정원, 주차시설, 술집, 화장실, 관광명소, 쇼핑 시설을 크롤링하는 함수 crawlNaverMap2()


In [4]:
#데이터 프레임생성, 각 열의 이름 선언 (장소명,관광요소 종류, 관광요소 이름)
df1=pd.DataFrame(columns=['place_name','tour_element','tour_element_name'])
df2=pd.DataFrame(columns=['place_name','review'])

### html path 저장
xpath가 바꼈을 때를 대비하여 코드에서 사용되는 html xpath들을 저장

In [70]:
#지도 크기 zoom-out
zoom_out='/html/body/app/layout/div[2]/div[2]/div[2]/maps-controller/dynamic-content-outlet[1]/control-widget/control-bottom-widget-holder/div/control-zoom/div/button[2]'

# 팝업창 닫기
popup_close='//*[@id="intro_popup_close"]/span'

#네이버 지도 홈으로 돌아가기
NmapHome='/html/body/app/layout/div[2]/div[1]/navbar/perfect-scrollbar/div/div[1]/div/ul/li[1]/a/span'

#네이버 지도 홈 검색카테고리
NmapHome_search='//*[@id="sidebar"]/navbar/perfect-scrollbar/div/div[1]/div/ul/li[2]/a/span[1]'

#네이버 지도 홈 검색창
NmapHome_searching='/html/body/app/layout/div[2]/div[2]/div[1]/app-base/search-box/div/div[2]/span'

#네이버 지도 검색 카테고리 검색창
NmapHome_searchbox='/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-box/div/div/div/input'

#검색 결과 없음
search_error='/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/h2'

#음식점
restaurants_path='/html/body/app/layout/div[2]/div[2]/div[1]/app-base/search-box/div/div[3]/div/around-here-card/ul/li[1]/a/span[1]'

#카페
cafes_path='/html/body/app/layout/div[2]/div[2]/div[1]/app-base/search-box/div/div[3]/div/around-here-card/ul/li[2]/a/span[1]'

#전체보기 
fullElem_path='/html/body/app/layout/div[2]/div[2]/div[1]/app-base/search-box/div/div[3]/div/around-here-card/ul/li[13]/button/span'

#숙박
lodges_path='/html/body/app/layout/div[2]/div[2]/div[1]/app-base/search-box/div/div[3]/div/around-here-card/ul/li[9]/a/span[1]'

#디저트
desserts_path='/html/body/app/layout/div[2]/div[2]/div[1]/app-base/search-box/div/div[3]/div/around-here-card/ul/li[7]/a/span[1]'

#엔터테인먼트
enter_path='/html/body/app/layout/div[2]/div[2]/div[1]/app-base/search-box/div/div[3]/div/around-here-card/ul/li[10]/a/span[1]'

#대중교통
trans_path='/html/body/app/layout/div[2]/div[2]/div[1]/app-base/search-box/div/div[3]/div/around-here-card/ul/li[4]/a/span[1]'

#자세히 보기 박스
detail='/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place[1]/div'

#리뷰 수
review_='/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/div/div[2]/div/div[1]/div[2]/div[2]/a/span'

#검색 지우기
delete='/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/search-box/div/div/button[2]'

#검색결과 페이지의 Next버튼
button_path='/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]'

#검색창
search_box='/html/body/div[1]/div/div[2]/div[2]/div/div/div[2]/header/div/nav/div[2]/div/div/form/input[1]'
            
#박스
box='/html/body/div[2]/div/div[2]/div/div/div/div/div[1]/div/div[1]/div/div[3]/div/div[1]/div/div[2]/div/div/div[1]/div/div/div/div[2]'

#리뷰버튼
review__='/html/body/div[2]/div/div[2]/div/div/div/div/div[1]/div/div[1]/div/div[3]/div/div[1]/div/div[2]/div/div/div[1]/div/div/div/div[2]/div[1]/div[2]/div/a'
              

### 함수 crawlNaverMap
> 음식점, 카페, 숙박, 디저트, 엔터, 대중교통 순으로 크롤링하는 함수

In [13]:
def crawlNaverMap():
    try:
            
#         #음식점
#         d.find_element_by_xpath(restaurants_path).click() #음식점 섹션 클릭
#         sleep(2)
#         i=0
            
#         while True:
#             try:
#                 r.append(d.find_elements_by_class_name('search_title_text')[i].text) #음식점 이름이 담겨있는 class를 가져와서 하나씩 리스트에 append

#                 if (i+1)%3==0: #화면에 3개씩 보이도록 스크롤
#                     ELEMENT = d.find_elements_by_class_name('search_title_text')[i+1] 
#                     d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
#                 i+=1
                        
#             except: break
            
#         d.find_element_by_xpath(NmapHome).click() #지도 홈 클릭
    

#         #카페
            
#         d.find_element_by_xpath(cafes_path).click() #카페 섹션 클릭
#         sleep(2) 
#         i=0
            
#         while True:
#             try:
#                 c.append(d.find_elements_by_class_name('search_title_text')[i].text)
        
#                 if (i+1)%3==0: #화면에 3개씩 보이도록 스크롤
#                     ELEMENT = d.find_elements_by_class_name('search_title_text')[i+1] 
#                     d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)                    
#                 i+=1
                        
#             except: break
           
#         d.find_element_by_xpath(NmapHome).click() #지도 홈 클릭     
#         d.find_element_by_xpath(fullElem_path).click() #전체보기     
                
#         #숙박
            
#         d.find_element_by_xpath(lodges_path).click()
#         sleep(2)
#         i=0
#         while True:
#             try:
#                 l.append(d.find_elements_by_class_name('search_title_text')[i].text)
                             
        
#                 if (i+1)%3==0: #화면에 3개씩 보이도록 스크롤
#                     ELEMENT = d.find_elements_by_class_name('search_title_text')[i+1] 
#                     d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        
#                 i+=1
#             except: break
             
#         d.find_element_by_xpath(NmapHome).click() #지도 홈 클릭
#         d.find_element_by_xpath(fullElem_path).click() #전체보기     
            
#         #디저트        
                    
#         d.find_element_by_xpath(desserts_path).click()
#         sleep(2)
#         i=0
#         while True:
#             try:
#                 dss.append(d.find_elements_by_class_name('search_title_text')[i].text)
        
#                 if (i+1)%3==0: #화면에 3개씩 보이도록 스크롤
#                     ELEMENT = d.find_elements_by_class_name('search_title_text')[i+1] 
#                     d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        
#                 i+=1
                        
#             except: break
                
#         d.find_element_by_xpath(NmapHome).click() #지도 홈 클릭
#         d.find_element_by_xpath(fullElem_path).click() #전체보기     
           
#         #엔터테인먼트
            
#         d.find_element_by_xpath(enter_path).click() 
        
#         #영화관, 코인노래방, 노래방, 볼링장, 당구장, 실내골프연습장 선택
#         d. find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/around-here-layout/around-here-home/div/div[1]/ul/li[2]/button').click()
#         d. find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/around-here-layout/around-here-home/div/div[1]/ul/li[3]/button').click()
#         d. find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/around-here-layout/around-here-home/div/div[1]/ul/li[4]/button').click()
#         d. find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/around-here-layout/around-here-home/div/div[1]/ul/li[5]/button').click()
#         d. find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/around-here-layout/around-here-home/div/div[1]/ul/li[6]/button').click()
#         d. find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/around-here-layout/around-here-home/div/div[1]/ul/li[7]/button').click()
        
#         sleep(2)
#         i=0
#         while True:
#             try:
#                 e.append(d.find_elements_by_class_name('search_title_text')[i].text)
        
#                 if (i+1)%3==0: #화면에 3개씩 보이도록 스크롤
#                     ELEMENT = d.find_elements_by_class_name('search_title_text')[i+1] 
#                     d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        
#                 i+=1
                        
#             except: break
        
#         d.find_element_by_xpath(NmapHome).click() #지도 홈 클릭
        
        #대중교통
        d.find_element_by_xpath(trans_path).click()

        #버스정류장
        sleep(2)
        i=0
        while True:
            try:
                t.append(d.find_elements_by_class_name('search_title_text')[i].text)
        
                if (i+1)%3==0: #화면에 3개씩 보이도록 스크롤
                    ELEMENT = d.find_elements_by_class_name('search_title_text')[i+1] 
                    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        
                i+=1
                        
            except: break
                
        #지하철역
        d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/around-here-layout/around-here-home/div/div[1]/ul/li[2]/button').click()
        sleep(2)
        i=0
        while True:
            try:
                t.append(d.find_elements_by_class_name('search_title_text')[i].text)
        
                if (i+1)%3==0: #화면에 3개씩 보이도록 스크롤
                    ELEMENT = d.find_elements_by_class_name('search_title_text')[i+1] 
                    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        
                i+=1
                        
            except: break
        
        #공공자전거
        d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/around-here-layout/around-here-home/div/div[1]/ul/li[3]/button').click()
        sleep(2)
        i=0
        while True:
            try:
                t.append(d.find_elements_by_class_name('search_title_text')[i].text)
        
                if (i+1)%3==0: #화면에 3개씩 보이도록 스크롤
                    ELEMENT = d.find_elements_by_class_name('search_title_text')[i+1] 
                    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        
                i+=1
                        
            except: break
                
    except:pass


In [79]:
#txt파일을 배열로 불러오기 
f = open('placeList.txt', 'r',encoding='UTF8')
place=[]
lines = f.readlines()
for line in lines:
    place.append(line)
    f.close()
    
# place=['읍내동 마애불상','경복궁']
search_kor=['음식점','카페','문화시설','숙박시설','공원','정원','주차시설','술집','화장실','관광명소','쇼핑']

In [80]:
print(len(place))

3830


### 함수 crawlNaverMap2
> 음식점, 카페, 문화시설, 숙박시설, 공원, 정원, 주차시설, 술집, 화장실, 관광명소, 쇼핑순으로 크롤링하는 함수

> 검색결과가 1개일 때 바로 그 검색결과의 detail로 들어가지기 때문에 그에대한 예외처리: <검색결과 버튼이 활성화되어있는 경우 그것을 클릭하고, 아니면(except) 평소대로 크롤링

In [56]:
def crawlNaverMap2():
    
    d.find_element_by_xpath(NmapHome_search).click()
    d.find_element_by_xpath(delete).click()
    
    #음식점
    try:
        elem=d.find_element_by_xpath(NmapHome_searchbox)
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[0])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(1)
        
        try:
            d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/search-box/div/div/button[1]').click()
            #검색결과가 1개인 경우 <검색결과 클릭하기
            
        except: pass
        
        #음식점 종류 클릭하기
        d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-tabs/div/div/div[2]/search-multi-filter/div/button').click()
        d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-tabs/div/div/div[2]/search-multi-filter/div[3]/div[1]/ul/li[2]/label').click()
        d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-tabs/div/div/div[2]/search-multi-filter/div[3]/div[1]/ul/li[3]/label').click()
        d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-tabs/div/div/div[2]/search-multi-filter/div[3]/div[1]/ul/li[4]/label').click()
        d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-tabs/div/div/div[2]/search-multi-filter/div[3]/div[1]/ul/li[5]/label').click()
        d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-tabs/div/div/div[2]/search-multi-filter/div[3]/div[1]/ul/li[6]/label').click()
        d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-tabs/div/div/div[2]/search-multi-filter/div[3]/div[1]/ul/li[7]/label').click()
        sleep(3)
        d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-tabs/div/div/div[2]/search-multi-filter/div[3]/div[4]/div/button[2]').click()       
        sleep(1)
        
        
        while True: #페이지 끝날때 까지
            for j in range(1,20):
                r.append(d.find_elements_by_class_name('search_title_text')[j].text)
                if j%3==0: #화면에 3개씩 보이도록 스크롤
                    ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
        
            next_button = d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
            if next_button.get_attribute("disabled")==None:
                next_button.click()
                sleep(2)
            else:
                #print('disabled')
                break                   
    except: pass
    
    #카페
    try:
        elem=d.find_element_by_xpath(NmapHome_searchbox)
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[1])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(1)
        
        try:
            d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/search-box/div/div/button[1]').click()
            #검색결과가 1개인 경우 <검색결과 클릭하기
            
        except: pass
        
        
        while True: #페이지 끝날때 까지
            for j in range(1,21):
                tag=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[2]/span').text
                if(tag=='카페'):
                    cafe=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                    c.append(cafe)
                    #print(park)
                        
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
                else:
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
            next_button = d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
            if next_button.get_attribute("disabled")==None:
                next_button.click()
                #print('click')
                sleep(2)
            else:
                #print('disabled')
                break                   
    except: pass
    
    
    #문화시설
    
    try:
        elem=d.find_element_by_xpath(NmapHome_searchbox)
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[2])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(1)
        
        try:
            d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/search-box/div/div/button[1]').click()
            #검색결과가 1개인 경우 <검색결과 클릭하기
            
        except: pass
        
        
        while True: #페이지 끝날때 까지
            for j in range(1,21):
                tag=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[2]/span').text
                if(tag=='문화시설'):
                    culture=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                    cul.append(culture)
                    #print(park)
                        
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
                else:
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
            next_button = d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
            if next_button.get_attribute("disabled")==None:
                next_button.click()
                #print('click')
                sleep(2)
            else:
                #print('disabled')
                break                   
    except: pass

    #숙박시설
    
    try:
        elem=d.find_element_by_xpath(NmapHome_searchbox)
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[3])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(1)
        
        try:
            d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/search-box/div/div/button[1]').click()
            #검색결과가 1개인 경우 <검색결과 클릭하기
            
        except: pass
           
        while True: #페이지 끝날때 까지
            for j in range(1,21):
                tag=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[2]/span').text
                if(tag=='호텔'):
                    hotel=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                    l.append(hotel)
                    #print(park)
                        
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
                else:
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
            next_button = d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
            if next_button.get_attribute("disabled")==None:
                next_button.click()
                #print('click')
                sleep(2)
            else:
                #print('disabled')
                break                   
    except: pass
        
        
   #공원
    try:
        elem=d.find_element_by_xpath(NmapHome_searchbox)
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[4])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(1)
        
        try:
            d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/search-box/div/div/button[1]').click()
            #검색결과가 1개인 경우 <검색결과 클릭하기
            
        except: pass
        
        while True: #페이지 끝날때 까지
            for j in range(1,21):
                tag=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[2]/span').text
                if(tag=='공원'):
                    park=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                    p.append(park)
                    #print(park)
                        
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
                else:
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
            next_button = d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
            if next_button.get_attribute("disabled")==None:
                next_button.click()
                #print('click')
                sleep(2)
            else:
                #print('disabled')
                break                   
    except: pass
                
    #정원
    try:
        elem=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-box/div/div/div/input')
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[5])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(1)
        
        try:
            d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/search-box/div/div/button[1]').click()
            #검색결과가 1개인 경우 <검색결과 클릭하기
            
        except: pass
        #print(idx+' 주변 '+search_kor[3]+'를 가져옵니다.') 
        while True: #페이지 끝날때 까지
            for j in range(1,21):
                tag=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[2]/span').text
                if(tag=='정원'):
                    garden=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                    g.append(garden)
                    #print(garden)
                        
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
                else:
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
            next_button = d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
            if next_button.get_attribute("disabled")==None:
                next_button.click()
                #print('click')
                sleep(2)
            else:
                #print('disabled')
                break                   
    except: pass
                
    #주차시설
    try:
        elem=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-box/div/div/div/input')
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[6])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(1)
        
        try:
            d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/search-box/div/div/button[1]').click()
            #검색결과가 1개인 경우 <검색결과 클릭하기
            
        except: pass
        #print(idx+' 주변 '+search_kor[3]+'를 가져옵니다.') 
        while True: #페이지 끝날때 까지
            for j in range(1,21):
                tag=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[2]/span').text
                if(tag=='주차장'):
                    parking=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                    pl.append(parking)
                    print(parking)
                        
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
                else:
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
            next_button = d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
            if next_button.get_attribute("disabled")==None:
                next_button.click()
                #print('click')
                sleep(2)
            else:
                #print('disabled')
                break                   
    except: pass
    #술집
    try:
        elem=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-box/div/div/div/input')
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[7])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(1)
        
        try:
            d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/search-box/div/div/button[1]').click()
            #검색결과가 1개인 경우 <검색결과 클릭하기
            
        except: pass
        #print(idx+' 주변 '+search_kor[3]+'를 가져옵니다.') 
        while True: #페이지 끝날때 까지
            for j in range(1,21):
                tag=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[2]/span').text
                if(tag=='이자카야'):
                    bar=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                    b.append(bar)
                    #print(bar)
                        
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
                else:
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
            next_button = d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
            if next_button.get_attribute("disabled")==None:
                next_button.click()
                #print('click')
                sleep(2)
            else:
                #print('disabled')
                break                   
    except: pass
                
    #화장실
    try:
        elem=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-box/div/div/div/input')
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[8])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(1)
        try:
            d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/search-box/div/div/button[1]').click()
            #검색결과가 1개인 경우 <검색결과 클릭하기
            
        except: pass
        #print(idx+' 주변 '+search_kor[3]+'를 가져옵니다.') 
        while True: #페이지 끝날때 까지
            for j in range(1,21):
                tag=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[2]/span').text
                if(tag=='화장실'):
                    toilet=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                    tl.append(toilet)
                    #print(toilet)
                        
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
                else:
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
            next_button = d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
            if next_button.get_attribute("disabled")==None:
                next_button.click()
                #print('click')
                sleep(2)
            else:
                #print('disabled')
                break                   
    except: pass
       
        
    #관광명소
    try:
        elem=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-box/div/div/div/input')
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[9])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(1)
        
        try:
            d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/search-box/div/div/button[1]').click()
            #검색결과가 1개인 경우 <검색결과 클릭하기
            
        except: pass
        
        
        sleep(2)
        #print(idx+' 주변 '+search_kor[3]+'를 가져옵니다.') 
        while True: #페이지 끝날때 까지
            for j in range(1,21):
                tag=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[2]/span').text
                if(tag=='관광명소'):
                    trip=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                    tr.append(trip)
                    #print(trip)
                        
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
                else:
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
            next_button = d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
            if next_button.get_attribute("disabled")==None:
                next_button.click()
                #print('click')
                sleep(2)
            else:
                #print('disabled')
                break                   
    except: pass
                
    #쇼핑
    try:
        elem=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-box/div/div/div/input')
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[10])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(1)
        
        try:
            d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-entry/entry-layout/entry-place/search-box/div/div/button[1]').click()
            #검색결과가 1개인 경우 <검색결과 클릭하기
            
        except: pass
        
        #print(idx+' 주변 '+search_kor[3]+'를 가져옵니다.') 
        while True: #페이지 끝날때 까지
            for j in range(1,21):
                tag=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[2]/span').text
                if(tag=='쇼핑'):
                    shopping=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                    s.append()
                   # print(shopping)
                        
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
                else:
                    if j%3==0: #화면에 3개씩 보이도록 스크롤
                        ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                        d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        #sleep(1)
            next_button = d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
            if next_button.get_attribute("disabled")==None:
                next_button.click()
                #print('click')
                sleep(2)
            else:
                #print('disabled')
                break                   
    except: pass
                

### 함수 add_dataframe
> 크롤링한 음식점,카페,디저트,숙소,교통,엔터,공원,정원,주차장,술집,화장실,관광명소,쇼핑 순서로 dataframe에 저장

In [27]:
def add_dataframe(n,idx,r,c,cul,l,t,p,g,pl,b,tl,tr,s):
    #n=한 장소당 데이터의 개수, 즉 index 수 
    
    for i in range(len(r)):
        df1.loc[n]=[idx,'음식점',r[i]]
        n+=1
        i+=1
   
    for i in range(len(c)):
        df1.loc[n]=[idx,'카페',c[i]]
        n+=1
        i+=1
        
    for i in range(len(cul)):
        df1.loc[n]=[idx,'문화시설',cul[i]]
        n+=1
        i+=1
        
    for i in range(len(l)):
        df1.loc[n]=[idx,'숙박시설',l[i]]
        n+=1
        i+=1
        
    for i in range(len(t)):
        df1.loc[n]=[idx,'대중교통',t[i]]
        n+=1
        i+=1
    
    for i in range(len(p)):
        df1.loc[n]=[idx,'공원',p[i]]
        n+=1
        i+=1
        
    for i in range(len(g)):
        df1.loc[n]=[idx,'정원',g[i]]
        n+=1
        i+=1
        
    for i in range(len(pl)):
        df1.loc[n]=[idx,'주차시설',pl[i]]
        n+=1
        i+=1
        
    for i in range(len(b)):
        df1.loc[n]=[idx,'술집',b[i]]
        n+=1
        i+=1
        
    for i in range(len(tl)):
        df1.loc[n]=[idx,'화장실',tl[i]]
        n+=1
        i+=1
        
    for i in range(len(tr)):
        df1.loc[n]=[idx,'관광명소',tr[i]]
        n+=1
        i+=1
        
    for i in range(len(s)):
        df1.loc[n]=[idx,'쇼핑',s[i]]
        n+=1
        i+=1
        
    return n #데이터의 개수 업데이트

### 문화유산 관광지 검색 main함수

In [61]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
header = {'User-Agent': ''}
options = Options()
d = webdriver.Chrome(executable_path="C:/Users/jmlee/Code/chromedriver_win32/chromedriver.exe",chrome_options=options) #웹앱을 테스트하는데 이용하는 프레임워크, webdriver라는 API를 통해 운영체제에 설치된 브라우저 제어 
d.set_window_size(1200, 800)
d.implicitly_wait(3)
d.get('https://map.naver.com/') 
sleep(2)
html=d.page_source
soup=BeautifulSoup(html, "html.parser")
sleep(1)

d.find_element_by_xpath(popup_close).click() #팝업 닫기
d.find_element_by_xpath(NmapHome_search).click() # 검색 항목 클릭
sleep(2)

C:\Users\jmlee\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [62]:
#페이지 불러오기
#<input _ngcontent-lqn-c13="" class="input_search ng-valid ng-dirty ng-touched" type="text" id="input_search1586413975870">
#리스트의 장소 검색

r=[] #음식점
c=[] #카페
l=[] #숙박시설
cul=[] #문화시설
t=[] #교통
                
p=[] #공원
g=[]  #정원
pl=[] #주차시설
b=[] #술집 
tl=[]#화장실
tr=[]#관광명소
s=[]#쇼핑
review=[] #리뷰수

n=0 #장소 당 데이터의 개수
count=0 #검색되는 장소의 개수

    
for idx in place:
    r=[] #음식점
    c=[] #카페
    l=[] #숙박
    cul=[] #문화시설
    t=[] #교통
    p=[] #공원
    g=[]  #정원
    pl=[] #주차시설
    b=[] #술집 
    tl=[]#화장실
    tr=[]#관광명소
    s=[]#쇼핑
    
    
    d.find_element_by_xpath(NmapHome).click() #지도 홈 클릭

    sleep(1)
    d.find_element_by_xpath(NmapHome_search).click() # 검색 항목 클릭
    elem= d.find_element_by_xpath(NmapHome_searchbox) #검색 항목의 검색창
    
    elem.clear()
    elem.send_keys(idx) #키워드 입력(경복궁,광화문 등등)
    elem.send_keys(Keys.ENTER) #엔터 치기
    sleep(1)
    #d.find_element_by_xpath(zoom_out).click()
    sleep(1)
    
    try: #에러 나는경우 > 검색 결과가 없음 >모든 리스트에 0넣기
        d.find_element_by_xpath(search_error) #에러메세지 paragraph
        #print(idx+'이/가 없습니다.')
        r=[0] #음식점
        c=[0] #카페,디저트
        l=[0] #숙소
        cul=[0] #문화시설
        t=[0] #대중교통
        
        p=[0] #공원
        g=[0]  #정원
        pl=[0] #주차시설
        b=[0] #술집 
        tl=[0]#화장실
        tr=[0]#관광명소
        s=[0]#쇼핑
        
        n=add_dataframe(n,idx,r,c,cul,l,t,p,g,pl,b,tl,tr,s) #데이터프레임에 추가
        #d.find_element_by_xpath(delete).click()
        sleep(1)
        
    except: #검색 결과 있음:에러메세지 출력되지 않음(except한 경우) > parsing 후 crawl 함수 실행   
        
        #리뷰 크롤링
        d.find_element_by_xpath(detail).click() #자세히보기
        a=d.find_element_by_xpath(review_).text #리뷰수 크롤링
        
        df2.loc[count]=[idx,a] #data frame에 리뷰수 저장
        count +=1
        
        #검색 크롤링
        
#         d.find_element_by_xpath(NmapHome).click() #지도 홈 클릭
#         crawlNaverMap()
        
        d.find_element_by_xpath(NmapHome).click() #지도 홈 클릭
        crawlNaverMap2()
        
        n=add_dataframe(n,idx,r,c,cul,l,t,p,g,pl,b,tl,tr,s) #데이터프레임에 추가
        
        #d.find_element_by_xpath(delete).click()
        
        sleep(1)
        

경복궁 주차장
경복궁주차장
안국역4번출구주차장
국립현대미술관서울주차장
조계사주차장
MYCJ유료주차장
정부서울청사주차장
서울지방경찰청주차장
행복민영주차장
정부서울청사주차장
정부서울청사주차장
정부서울청사주차장
정부서울청사주차장
서울지방경찰청주차장
삼청공용주차장
국원산업
종로구청주차장
윌슨파킹코리아 센터플레이스 주차장
종로구청주차장
맥서브주차장
감사원주차장
서울적십자병원주차장
강북삼성병원주차장
유료주차장
학교법인이화학원 주차장


In [67]:
df2

,place_name,review
0,경복궁,8740


## 트리바이저

In [75]:
def tripadvisior():
    header = {'User-Agent': ''}
    d = webdriver.Chrome(executable_path="C:/Users/jmlee/Code/chromedriver_win32/chromedriver.exe") #웹앱을 테스트하는데 이용하는 프레임워크, webdriver라는 API를 통해 운영체제에 설치된 브라우저 제어 
    d.implicitly_wait(3)
    d.get('https://www.tripadvisor.co.kr/') 
    sleep(2)

    html=d.page_source
    soup=BeautifulSoup(html, "html.parser")
    sleep(1)
    
    elem=d.find_element_by_xpath(search_box) #검색창
    elem.send_keys(idx) #place전달
    d.find_element_by_xpath(search_box).send_keys(Keys.ENTER) #엔터 치기
    sleep(2)  
    try:
        review=d.find_element_by_xpath(review__).text
        review_=re.sub(',','',review)
        numbers = re.findall("\d+", review_)[0]
        print(numbers)
        df2.loc[count]=[idx,numbers] 
    except:
        df2.loc[count]=[idx,0]
        print('no exist')

In [76]:
tripadvisior()

10646


In [77]:
df2

,place_name,review
0,경복궁,8740
1,경복궁,10646


## 데이터셋 csv파일로 얻기

In [78]:
df1.to_csv("Dongjak_Bigdata.csv",mode='w') 
df2.to_csv("Dongjak_Bigdata_review.csv",mode='w')